# train LG3:   LG3 and preprocessor3
trained on imbalanced CJP (clinvar + 50%( justhusky + KI pathogenic)) with 12 features


In [2]:
import gzip
import pandas as pd
import numpy as np
from joblib import dump, load

import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import BinaryEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

/Users/nancy/miniconda3/envs/bioinfo/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# preprocess

data: Clinvar + 50% (justhusky + KI pathogenic)

In [3]:
# get train data: clin just newpatho
df_train_merge=pd.read_csv('/Users/nancy/Desktop/RS_projects/data/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/01_train_merge_ClinvarJustPatho_add29features.csv')
df_train_merge

,CLNSIG,CADD,AF_TGP,Frq,IMPACT,GNOMADAF_popmax,Hom,ORIGIN,SPIDEX,SWEGENAF,...,GERP++_RS,REVEL_score,phastCons100way_vertebrate,phyloP100way_vertebrate,LoFtool,pLI_gene_value,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL
0,Uncertain_significance,26.60,NaN,NaN,MODERATE,NaN,NaN,1.0,1.763,NaN,...,3.600,0.347,1.000,7.343,NaN,0.00,0.01,0.00,0.09,0.00
1,Likely_benign,13.42,NaN,NaN,LOW,NaN,NaN,1.0,-1.030,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.02,0.03,0.00
2,Likely_benign,31.00,NaN,0.00056,MODERATE,0.000195,NaN,1.0,-0.910,NaN,...,4.520,0.429,1.000,5.538,NaN,0.00,0.00,0.01,0.06,0.00
3,Uncertain_significance,28.20,NaN,NaN,MODERATE,NaN,NaN,1.0,0.814,NaN,...,4.520,0.458,1.000,3.691,NaN,0.00,0.00,0.00,0.03,0.02
4,Likely_benign,11.38,NaN,NaN,LOW,NaN,NaN,1.0,0.992,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.01,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757032,Pathogenic,25.40,NaN,NaN,MODERATE,NaN,NaN,NaN,-1.648,NaN,...,3.050,0.716,1.000,7.489,0.151,1.00,0.00,0.00,0.00,0.00
3757033,Pathogenic,21.30,NaN,0.00140,MODERATE,NaN,NaN,NaN,NaN,0.0005,...,1.560,0.219,0.009,0.707,NaN,0.18,0.03,0.00,0.00,0.00
3757034,Pathogenic,24.00,NaN,0.00014,MODERATE,NaN,NaN,NaN,NaN,NaN,...,5.290,0.287,0.998,3.589,0.739,0.00,0.00,0.00,0.00,0.00
3757035,Pathogenic,16.88,NaN,0.00350,MODERATE,0.001555,NaN,NaN,-1.149,0.0015,...,1.440,0.025,0.320,0.737,0.143,0.94,0.00,0.00,0.00,0.00


In [5]:
""" 1. choose featuresV1"""

featureV1 = ['CLNSIG',
'CADD',
'Frq',
'GNOMADAF_popmax',
'Consequence',
'BIOTYPE',
'PolyPhen',
'REVEL_score',
'pLI_gene_value',
'SpliceAI_pred_DS_AG',
'SpliceAI_pred_DS_AL',
'SpliceAI_pred_DS_DG',
'SpliceAI_pred_DS_DL']         # keep CLNSIG,

df_1 = df_train_merge[featureV1]



""" 2. only remove missing target """

drop_nan_features = ['CLNSIG']

df_1 = df_1.dropna(subset=drop_nan_features)

df_1 


,CLNSIG,CADD,Frq,GNOMADAF_popmax,Consequence,BIOTYPE,PolyPhen,REVEL_score,pLI_gene_value,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL
0,Uncertain_significance,26.60,NaN,NaN,missense_variant,protein_coding,probably_damaging,0.347,0.00,0.01,0.00,0.09,0.00
1,Likely_benign,13.42,NaN,NaN,synonymous_variant,protein_coding,NaN,NaN,0.00,0.00,0.02,0.03,0.00
2,Likely_benign,31.00,0.00056,0.000195,missense_variant,protein_coding,probably_damaging,0.429,0.00,0.00,0.01,0.06,0.00
3,Uncertain_significance,28.20,NaN,NaN,missense_variant,protein_coding,probably_damaging,0.458,0.00,0.00,0.00,0.03,0.02
4,Likely_benign,11.38,NaN,NaN,synonymous_variant,protein_coding,NaN,NaN,0.00,0.00,0.00,0.01,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757032,Pathogenic,25.40,NaN,NaN,missense_variant,protein_coding,probably_damaging,0.716,1.00,0.00,0.00,0.00,0.00
3757033,Pathogenic,21.30,0.00140,NaN,missense_variant,protein_coding,probably_damaging,0.219,0.18,0.03,0.00,0.00,0.00
3757034,Pathogenic,24.00,0.00014,NaN,missense_variant,protein_coding,probably_damaging,0.287,0.00,0.00,0.00,0.00,0.00
3757035,Pathogenic,16.88,0.00350,0.001555,missense_variant,protein_coding,benign,0.025,0.94,0.00,0.00,0.00,0.00


In [6]:
""" 3. deal with target feature """
# """  CLNSIG: tansfer to 4 classes__ benign,pathogenic,uncertain,others
# NOTE:
# (only count name before |)   explanation-- https://www.ncbi.nlm.nih.gov/clinvar/docs/clinsig/#clinsig_scv

# 1. pathogenic: Pathogenic, Pathogenic/Likely_pathogenic, Likely_pathogenic
# 2. benign: Likely_benign, Benign, Benign/Likely_benign
# 3. uncertain: Uncertain_significance, 
# 4. others: Uncertain_risk_allele, risk_factor, protective, other, not_provided, Likely_risk_allele, drug_response, Conflicting_interpretations_of_pathogenicity, confers_sensitivity, associatio, Affects

df_1 = df_1.copy()
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Uncertain_significance|_risk_factor', 'Uncertain_significance|_other', 'Uncertain_significance|_drug_response','Uncertain_significance|_association','Uncertain_significance|_Affects','Uncertain_significance'], 'uncertain')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Pathogenic|_risk_factor','Pathogenic|_protective','Pathogenic|_other','Pathogenic|_drug_response|_other','Pathogenic|_drug_response','Pathogenic|_confers_sensitivity','Pathogenic|_association','Pathogenic|_Affects','Pathogenic/Likely_risk_allele','Pathogenic/Likely_pathogenic|_risk_factor','Pathogenic/Likely_pathogenic|_other','Pathogenic/Likely_pathogenic|_drug_response','Pathogenic/Likely_pathogenic','Pathogenic','Likely_pathogenic|_risk_factor','Likely_pathogenic|_other','Likely_pathogenic|_drug_response','Likely_pathogenic|_association','Likely_pathogenic|_Affects','Likely_pathogenic,_low_penetrance','Likely_pathogenic'], 'pathogenic')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Likely_benign|_risk_factor','Likely_benign|_other','Likely_benign|_drug_response|_other','Likely_benign','Benign|_risk_factor','Benign|_protective','Benign|_other','Benign|_drug_response','Benign|_confers_sensitivity','Benign|_association|_confers_sensitivity','Benign|_association','Benign/Likely_benign|_risk_factor','Benign/Likely_benign|_other|_risk_factor','Benign/Likely_benign|_other','Benign/Likely_benign|_drug_response|_other','Benign/Likely_benign|_drug_response','Benign/Likely_benign|_association','Benign/Likely_benign','Benign'], 'benign')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Uncertain_risk_allele|_risk_factor','Uncertain_risk_allele','risk_factor','protective','protective|_risk_factor','other', 'not_provided','Likely_risk_allele','drug_response|_risk_factor','drug_response|_other','drug_response','Conflicting_interpretations_of_pathogenicity|_risk_factor','Conflicting_interpretations_of_pathogenicity|_protective','Conflicting_interpretations_of_pathogenicity|_other|_risk_factor','Conflicting_interpretations_of_pathogenicity|_other','Conflicting_interpretations_of_pathogenicity|_drug_response|_other','Conflicting_interpretations_of_pathogenicity|_drug_response','Conflicting_interpretations_of_pathogenicity|_association|_risk_factor','Conflicting_interpretations_of_pathogenicity|_association','Conflicting_interpretations_of_pathogenicity','confers_sensitivity','association|_risk_factor','association_not_found','association','Affects|_risk_factor','Affects|_association','Affects'], 'others')

# only keep benign & pathogenic
df_1 = df_1.loc[df_1['CLNSIG'].isin(['benign','pathogenic'])]

# label encoder
map_clnsig={'pathogenic':1,'benign':0}
df_1['CLNSIG']=df_1['CLNSIG'].map(lambda s: map_clnsig.get(s) if s in map_clnsig else s)
df_1["CLNSIG"].unique()

array([0, 1])

In [7]:
"""4.2  keep imbalance classes """

# split

X, y = df_1.iloc[:, 1:], df_1.iloc[:, 0]
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                    test_size=0.3,
                    random_state=42,
                    stratify=y)

In [8]:
""" 5. features engineering: transformer"""
## only for training data, no care about target 'CLNSIG', total 29 features


# ## numerical feature group 1 : replace missing with mean;  scaling/standardization

# numeric_feature_1 = ['MES-SWA_acceptor_alt','MES-SWA_acceptor_diff','MES-SWA_donor_alt','MES-SWA_donor_diff','MaxEntScan_alt','MaxEntScan_diff','GERP++_RS',
#     'phastCons100way_vertebrate','phyloP100way_vertebrate']



## numerical feature group 2 : replace missing with 0;  scaling/standardization
numeric_feature_2 = ['CADD','Frq', 'GNOMADAF_popmax','pLI_gene_value','SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG','SpliceAI_pred_DS_DL']



## numerical feature group 3: add missing indicator; replace missing with 0 and ; scaling/standardization
numeric_feature_3 = ['REVEL_score','SpliceAI_pred_DS_AG']



## categorical ordinal feature group 4: add missing indicator for ['PolyPhen']; replace missing with 'missing_value' string; OrdinalEncoder
categori_oridinal_feature_4 =['Consequence','PolyPhen']

# ordinal features' order: left to right will be from 0 to length-1
with open('variant_consequences.txt','r') as f:             # get the 'variant_consequences order' lst: descending severity
    order_conseq=[line.rstrip("\n") for line in f] 
    order_conseq.reverse()

ordinal_features = [
    'Consequence',
    'IMPACT',
    'SIFT',
    'PolyPhen']
# 0 is the missing value will be replaced by
ordinal_ordering = [
    order_conseq,
    ['unknown','benign','possibly_damaging','probably_damaging']
    ]


## categorical nominal feature group 5: replace missing with ''missing_value' in ['BIOTYPE'];  BinaryEncoder
categori_nominal_feature_5 = ['BIOTYPE']  #,'BIOTYPE'



# numeric_feature_1_transformer =make_pipeline(
#     SimpleImputer(strategy='mean'),
#     StandardScaler()
# )

numeric_feature_2_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=0),
    StandardScaler()
)

numeric_feature_3_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=0),
    StandardScaler()
)

categori_oridinal_feature_4_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=None),  # If None, fill_value will be 0 when imputing numerical data and “missing_value” for strings
    OrdinalEncoder(categories=ordinal_ordering, handle_unknown='use_encoded_value', unknown_value = -1)       # the unknown 'missing_value' string will be enocoded as '-1'
)

categori_oridinal_feature_5_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=None),  
    BinaryEncoder()                                # add new columns, throw the original columns automaticly, 
)


preprocessor = make_column_transformer(
    # (numeric_feature_1_transformer, numeric_feature_1),
    (numeric_feature_2_transformer, numeric_feature_2 ),
    (MissingIndicator(), numeric_feature_3 + categori_oridinal_feature_4[1:]),   # add new missing indicators for feature3 + ['PolyPhen'] without originial columns,  here: total add 3 indicator columns
    (numeric_feature_3_transformer, numeric_feature_3),  
    (categori_oridinal_feature_4_transformer, categori_oridinal_feature_4),
    (categori_oridinal_feature_5_transformer, categori_nominal_feature_5),                                                    
    # (categori_oridinal_feature_5_transformer, categori_nominal_feature_5[1:])   # for checking: how many ori and biotype derived features seperately; must use [1:] formate, and see the `'BIOTYPE` first, and then can get `origin`
    # remainder='passthrough'                                                   # so here for `BIOTYPE` derived new: 6
)

## total features after preprocessor: 20


In [9]:
### check the features names and number after preprocessing

## after all steps: tatol 19 ouputs,  so  derived BIOTYPE should have 19-14 = 5 
check_X_pre = preprocessor.fit_transform(X_train)  

pd.DataFrame(check_X_pre)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.573702,1.359218,-1.105721,-0.518826,-0.097007,-0.085289,-0.095069,1.0,0.0,1.0,-0.152106,-0.096912,13.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.526005,1.517659,0.283417,-0.518826,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.542452,0.313204,-0.302264,-0.518826,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.606850,0.100155,0.091827,-0.492174,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,13.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.006263,2.107580,0.470206,2.146404,-0.097007,-0.085289,-0.095069,1.0,0.0,1.0,-0.152106,-0.096912,13.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155278,-0.677954,0.257826,-0.197959,-0.518826,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0
2155279,2.813601,-1.212747,-1.105721,-0.518826,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,34.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
2155280,-0.340274,-0.396702,0.862552,-0.518826,-0.097007,1.360244,0.355621,1.0,0.0,1.0,-0.152106,-0.096912,13.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
2155281,-0.573070,0.156301,-0.629582,-0.518826,-0.097007,-0.085289,-0.095069,1.0,1.0,1.0,-0.152106,-0.096912,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# steps before  5_transformer   : 14 output

col1 = numeric_feature_2
col2 = ['miss_REVEL_score', 'miss_SpliceAI_pred_DS_AG','miss_PolyPhen']
col3 = numeric_feature_3 + categori_oridinal_feature_4
col4 = ['BIOTYPE_0','BIOTYPE_1','BIOTYPE_2','BIOTYPE_3','BIOTYPE_4','BIOTYPE_5']
cols = col1 + col2 + col3 +col4      ## get all columns's names after preprocessing
len(cols)

20

# train

In [11]:
## fit and real test score
lr = LogisticRegression(max_iter=5000, C=10, random_state= 42)

pipeline = make_pipeline(preprocessor, lr) 
pipeline.fit(X_train, y_train)              # preprocessor and lr both fitted by X_train here!

print(f"accurate on test set: {pipeline.score(X_train, y_train):.4f}")
print(f"accurate on test set: {pipeline.score(X_test, y_test):.4f}")

accurate on test set: 0.9943
accurate on test set: 0.9944


### save LG3 and preprocessor3

In [44]:
dump(lr,'/models/LG3_12feats_CJP.joblib')                        #  02_12feats_ClinJustPatho5-5_LG.joblib

dump(preprocessor,'/models/preprocessor3_12feats_CJP.joblib')    # 02_12feats_ClinJustPatho5-5_preprocessor.joblib

['/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/01_featureV1/retrain_justhusky_newpatho/models/02_12feats_ClinJustPatho5-5_preprocessor.joblib']